In [1]:
import numpy as np
from scipy import optimize
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time

import glob
import os


# Functions

In [2]:
def station_to_city(station_name):
    if "East_Palo_Alto" in station_name:
        city_name = "East Palo Alto"
    elif "Palo_Alto" in station_name:
        city_name = "Palo Alto"
    elif "Menlo_Park" in station_name:
        city_name = "Menlo Park"
    elif "Redwood_City" in station_name:
        city_name = "Redwood City"
    elif "Belmont" in station_name:
        city_name = "Belmont"
    elif "San_Mateo" in station_name:
        city_name = "San Mateo"
    elif "Burlingame" in station_name:
        city_name = "Burlingame"
    elif "Millbrae" in station_name:
        city_name = "Millbrae"
    else:
        assert 1 == 0, "There should be no other case."
    return city_name

# Latency Function Inference

In [ ]:
directory_path_speed = '../data/pems_speed___101_N_Sep_2024/'
directory_path_flow = '../data/pems_flow___101_N_Sep_2024/'

speed_file_list = [f for f in os.listdir(directory_path_speed) \
                  if os.path.isfile(os.path.join(directory_path_speed, f)) and f[-1] == 'x' and f[0] != "~"]
speed_file_list.sort()

flow_file_list = [speed_file[:4] + speed_file[10:] for speed_file in speed_file_list]
# flow_file_list

In [ ]:
timestamp_list = []

date_list = ['3', '4', '5', '6', '9', '10', '11', '12', '13', '16',\
             '17', '18', '19', '20', '23', '24', '25', '26', '27']

for date in date_list:
    for timestamp in range(24):
        timestamp_list += ['9/' + date + '/2024 ' + str(timestamp) + ':00']

# timestamp_list

In [ ]:
# print("speed_file_list[0]:", speed_file_list[0])
# directory_path = "../data/pems_speed___101_N_Sep_2024/"
# df_speed_file = pd.read_excel(directory_path + speed_file_list[0], index_col=0)

In [ ]:
speed_file_temp = speed_file_list[0]
print("speed_file_temp:", speed_file_temp)
# df_speed_file_temp = pd.read_excel('../data/pems_speed___101_N_Sep_2024/001_speed___402376_Palo_Alto___main.xlsx', index_col=0)

df_speed_file_temp = pd.read_excel(directory_path_speed + speed_file_temp, index_col=0)

df_speed_file_temp.loc["9/3/2024 0:00", "Speed (mph)"]
# df_speed_file_temp.loc["9/3/2024 0:00", "Speed (mph)"]


In [ ]:
start_time = time.time()

# directory_path = "../data/pems_speed___101_N_Sep_2024/"
city_list = ["Palo Alto", "East Palo Alto", "Redwood City", "Belmont", "San Mateo", "Burlingame", "Millbrae"]

speed_dict = {}

for speed_file in speed_file_list:
    speed_dict[speed_file[:-5]] = {}
    
    df_speed_file = pd.read_excel(directory_path_speed + speed_file, index_col=0)

    for timestamp in timestamp_list:
        speed_dict[speed_file[:-5]][timestamp] = df_speed_file.loc[timestamp, "Speed (mph)"]
#         df_speed_file_temp.loc["9/3/2024 0:00", "Speed (mph)"]

end_time = time.time()
print("Time:", end_time - start_time)



In [ ]:
start_time = time.time()

num_gp_lanes = 4

flow_dict = {}

for flow_file in flow_file_list:
    flow_dict[flow_file[:-5]] = {}
    
    df_flow_file = pd.read_excel(directory_path_flow + flow_file, index_col=0)
    
    for timestamp in timestamp_list:
        flow_dict[flow_file[:-5]][timestamp] = df_flow_file.loc[timestamp, "Flow (Veh/Hour)"] / num_gp_lanes
        
end_time = time.time()
print("Time:", end_time - start_time)

# flow_file_list


In [ ]:
directory_path_latency = '../data/pems_latency_inference/'
distance_file = "distances_between_sensors.xlsx"

df_distance_file = pd.read_excel(directory_path_latency + distance_file, index_col=0)
distance_dict = {}

# df_distance_file.loc["Data Category", "0"]
# df_distance_file.loc["Start City", 10]

for sub_edge in list(df_distance_file.columns):
    distance_dict[sub_edge] = {}
    distance_dict[sub_edge]["Start City"] = df_distance_file.loc["Start City", sub_edge]
    distance_dict[sub_edge]["Start Station"] = df_distance_file.loc["Start Station", sub_edge]
    distance_dict[sub_edge]["End Station"] = df_distance_file.loc["End Station", sub_edge]
    distance_dict[sub_edge]["Distance"] = df_distance_file.loc["Distances Between Sensors (miles)", sub_edge]


In [ ]:
flow_time_dict = {}
flow_threshold = 100

for city in city_list:
    flow_time_dict[city] = []
    
    station_speed_list = []
    station_speed_list += [distance_dict[sub_edge]["Start Station"] for sub_edge in distance_dict.keys() \
                           if distance_dict[sub_edge]["Start City"] == city]
    station_speed_list += [distance_dict[sub_edge]["End Station"] for sub_edge in distance_dict.keys() \
                           if distance_dict[sub_edge]["Start City"] == city]
    station_speed_list = list(set(station_speed_list))
    
    station_flow_list = [station_speed[:3] + station_speed[9:] for station_speed in station_speed_list]
    
    sub_edge_list = [key for key in distance_dict.keys() if distance_dict[key]["Start City"] == city]
    
#     print(station_speed_list)
#     print()
    
    for timestamp in timestamp_list:
        # TODO: Compute flow array
#         flow_array = np.array([flow_dict[station][timestamp] for station in station_flow_list \
#                                if flow_dict[station][timestamp] >= flow_threshold])
        flow_array = np.array([flow_dict[station][timestamp] for station in station_flow_list])
        average_flow = np.mean(flow_array)
        
        travel_time = sum([distance_dict[sub_edge]["Distance"] * 2 \
                           / (speed_dict[distance_dict[sub_edge]["Start Station"]][timestamp] \
                              + speed_dict[distance_dict[sub_edge]["Start Station"]][timestamp]) \
                           for sub_edge in sub_edge_list])
        
#         # TODO: Compute travel time
        
        flow_time_dict[city] += [(average_flow, travel_time)]
        
# flow_time_dict["Millbrae"]

In [ ]:
def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])

def piecewise_linear_simplified(x, x0, y0, k):
    return np.piecewise(x, [x < x0], [lambda x:y0, lambda x:k*x + y0-k*x0])

# p , e = optimize.curve_fit(piecewise_linear, x, y)

In [ ]:
latency_params = {}
num_flow_potential_bends = 100

for city in city_list:
    flows = np.array([flow_time_tuple[0] for flow_time_tuple in flow_time_dict[city]], dtype = float)
    times = np.array([flow_time_tuple[1] for flow_time_tuple in flow_time_dict[city]])
    
    max_flow = np.max(flows)
    min_flow = np.min(flows)
    
    flow_potential_bend_list = min_flow + (max_flow - min_flow) * list(range(num_flow_potential_bends))[1:]
    best_flow_time_slope_error = []
    
    for flow_potential_bend in flow_potential_bend_list:
        
        flow_times_at_constant = [pair for pair in flow_time_dict[city] if pair[0] <= flow_potential_bend]
        flow_times_at_slope = [pair for pair in flow_time_dict[city] if pair[0] > flow_potential_bend]
        
        # TODO: Set up optimization problem.
        
        # Variables:
        y = cp.Variable(1)
        m = cp.Variable(1)
        
        # Objectives:
        func = 0.0
#         func += cp.sum([])
#         func += cp.sum([])
        
        # Constraints:
        
        
        # Solve problem:
        prob = cp.Problem(objective, constraints)
        result = prob.solve()
        
        # Print solution:
        
        
#         if best_flow_time_slope_error == []:
            
            
        
        
    
#     print("flows:", flows)
#     print()
#     print("times:", times)
#     print()
#     print()
    
    # params - x of junction point, y of junction point, slope of 1st segment, slope of 2nd segment
    params, _ = optimize.curve_fit(piecewise_linear_simplified, flows, times)
    
    latency_params[city] = list(params)



In [ ]:
latency_params

In [ ]:
# flows_temp = [flow_time_tuple[0] for flow_time_tuple in flow_time_dict["Palo Alto"]]
# times_temp = [flow_time_tuple[1] for flow_time_tuple in flow_time_dict["Palo Alto"]]

# flows_temp = [flow_time_tuple[0] for flow_time_tuple in flow_time_dict["San Mateo"]]
# times_temp = [flow_time_tuple[1] for flow_time_tuple in flow_time_dict["San Mateo"]]

flows_temp = [flow_time_tuple[0] for flow_time_tuple in flow_time_dict["Millbrae"]]
times_temp = [flow_time_tuple[1] for flow_time_tuple in flow_time_dict["Millbrae"]]

plt.plot(flows_temp, times_temp, "o")


In [ ]:
# TODO: 

# Create an dict of average speeds.
# Create a dict, from distances_between_sensors.xlsx, of distances betwen sub-edges in that city.

# For each city:
# For each date / hour timestap:
# Find flow vector along various sub-edges of that city at that timestamp. If each flow vector < 100, \
# ignore that timestamp. Otherwise, compute average flow for that city at that timestamp, and divide by num_gp_lanes.
# Compute the time necessary to travel through that city at that timestamp, \
# and add it to the (flow, time) tuple / array list for that city.


In [ ]:
# flow_thresh_low = 100


In [ ]:
directory_path = '../data/data_income_percentage_VoT/'
df_data = pd.read_excel(directory_path + 'data_cities_od_VoTs_demands_1.csv')

dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])


# Scratch Work - Piecewise Linear Function

In [ ]:
# Link:
# https://stackoverflow.com/questions/29382903/how-to-apply-piecewise-linear-fit-in-python

x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ,11, 12, 13, 14, 15], dtype=float)
y = np.array([5, 7, 9, 11, 13, 15, 28.92, 42.81, 56.7, 70.59, 84.47, 98.36, 112.25, 126.14, 140.03])

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])

p , e = optimize.curve_fit(piecewise_linear, x, y)
xd = np.linspace(0, 15, 100)
plt.plot(x, y, "o")
plt.plot(xd, piecewise_linear(xd, *p))

In [ ]:
p_list = list(p)
# p
# e

In [ ]:
p_list

# Scratch Work